In [1]:
import sqlalchemy as alchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, Float, BigInteger
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
NOMBRE_DB = "tp3punto3"
USUARIO_DB = 'postgres' #gaston
PASSWORD_DB = 'postgres' #gastonm1
url_db = 'postgresql+psycopg2://%s:%s@localhost:5432/%s'%(USUARIO_DB, PASSWORD_DB,'')

engine = alchemy.create_engine(url_db, echo=True)
try:
    conn = engine.connect()
    conn.execute("commit")
    conn.execute("create database " + NOMBRE_DB)
    conn.close()
except Exception as e:
    print str(e)
url_db = 'postgresql+psycopg2://%s:%s@localhost:5432/%s'%(USUARIO_DB, PASSWORD_DB, NOMBRE_DB)
engine = alchemy.create_engine(url_db, echo=True)

Base = declarative_base()

2017-09-05 18:30:24,739 INFO sqlalchemy.engine.base.Engine select version()
2017-09-05 18:30:24,741 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,744 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-09-05 18:30:24,744 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,749 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-09-05 18:30:24,750 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,752 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-09-05 18:30:24,755 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,760 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-09-05 18:30:24,761 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,764 INFO sqlalchemy.engine.base.Engine commit
2017-09-05 18:30:24,767 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,769 INFO sqlalchemy.engine.base.Engine create databas

In [3]:
class Ciudad(Base):
    __tablename__ = 'Ciudad'
    
    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    producto = Column(Integer)


In [4]:
Base.metadata.create_all(engine)

2017-09-05 18:30:24,975 INFO sqlalchemy.engine.base.Engine select version()
2017-09-05 18:30:24,977 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,985 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-09-05 18:30:24,991 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:24,998 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-09-05 18:30:25,002 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:25,008 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-09-05 18:30:25,012 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:25,020 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-09-05 18:30:25,025 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:30:25,032 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

## 1) Tomar datos de un archivo csv, de json y una db SQL

### Lectura SQL

In [5]:
df_ciudades = pd.read_sql("select * from \"Ciudad\"", engine, index_col='id')
df_ciudades.head(5)

2017-09-05 18:30:25,064 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2017-09-05 18:30:25,065 INFO sqlalchemy.engine.base.Engine {'name': u'select * from "Ciudad"'}
2017-09-05 18:30:25,069 INFO sqlalchemy.engine.base.Engine select * from "Ciudad"
2017-09-05 18:30:25,071 INFO sqlalchemy.engine.base.Engine {}


,nombre,producto
id,,


### Lectura CSV

In [6]:
datos_producto = 'data/MOCK_DATA-tp3-punto3-bd-producto.csv'
df_producto = pd.DataFrame.from_csv(datos_producto,sep=',',index_col='id')
df_producto.head(5)

,nombre,precio,localidad,tipoProducto
id,,,,
1,Nathan,$852.82,2,8
2,Abramo,$249.03,2,7
3,Ethelyn,$769.93,4,2
4,Von,$32.94,7,2
5,Latrena,$762.18,5,10


### Lectura JSON

In [7]:
datos_tipo_producto = 'data/MOCK_DATA-tp3-punto3-json2-tipoProducto.json'
df_tipoProducto = pd.read_json(datos_tipo_producto)
df_tipoProducto= df_tipoProducto.set_index('id')
df_tipoProducto.head(5)

,nombre
id,
1,Wells Fargo & Company
2,Matlin & Partners Acquisition Corporation
3,Nabors Industries Ltd.
4,"American Financial Group, Inc."
5,Axis Capital Holdings Limited


## 2) De los tres DataFrames realizar join, merge, concat, append, con una clave a eleccion.

In [8]:
frames = [df_ciudades, df_producto, df_tipoProducto]

In [9]:
resultConcatenacion = pd.concat(frames)
resultConcatenacion.head(5)

,localidad,nombre,precio,producto,tipoProducto
id,,,,,
1,2.0,Nathan,$852.82,NaN,8.0
2,2.0,Abramo,$249.03,NaN,7.0
3,4.0,Ethelyn,$769.93,NaN,2.0
4,7.0,Von,$32.94,NaN,2.0
5,5.0,Latrena,$762.18,NaN,10.0


In [10]:
resultMerge = pd.merge(df_producto, df_ciudades)
resultMerge.head(5)

,nombre,precio,localidad,tipoProducto,producto


In [11]:
datos_producto2 = 'data/MOCK_DATA-tp3-punto3-bd-producto2.csv'
df_producto2 = pd.DataFrame.from_csv(datos_producto2,sep=',',index_col='id')
df_producto2.head(5)

,nombre,precio,localidad,tipoProducto
id,,,,
1,Ferd,$188.24,3,8
2,Nollie,$584.78,3,8
3,Kelly,$887.78,8,7
4,Vick,$651.07,5,4
5,Rosaleen,$131.19,9,7


In [12]:
resultAppend = df_producto.append(df_producto2)
resultAppend.head(5)

,nombre,precio,localidad,tipoProducto
id,,,,
1,Nathan,$852.82,2,8
2,Abramo,$249.03,2,7
3,Ethelyn,$769.93,4,2
4,Von,$32.94,7,2
5,Latrena,$762.18,5,10


In [13]:
resultadoJoinAll=pd.merge(df_producto, df_producto2, how='outer')
resultadoJoinAll.head(5)

,nombre,precio,localidad,tipoProducto
0,Nathan,$852.82,2,8
1,Abramo,$249.03,2,7
2,Ethelyn,$769.93,4,2
3,Von,$32.94,7,2
4,Latrena,$762.18,5,10


## 3) Guardar en una db SQL, la interseccion de los datos obtenidos en el punto anterior.

In [15]:
#DataFrame.to_sql(name, con[, flavor, ...])	Write records stored in a DataFrame to a SQL database.
resultadoJoinAll.to_sql('resultadoJoinAll', engine,if_exists='replace')
resultAppend.to_sql('resultadoAppend', engine, if_exists='replace')
resultConcatenacion.to_sql('resultadoConcatenacion', engine, if_exists='replace')
resultMerge.to_sql('resultadoMerge', engine, if_exists='replace')

2017-09-05 18:31:50,768 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2017-09-05 18:31:50,769 INFO sqlalchemy.engine.base.Engine {'name': u'resultadoJoinAll'}
2017-09-05 18:31:50,772 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2017-09-05 18:31:50,773 INFO sqlalchemy.engine.base.Engine {'name': u'resultadoJoinAll'}
2017-09-05 18:31:50,776 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind = 'r'
2017-09-05 18:31:50,777 INFO sqlalchemy.engine.base.Engine {'schema': u'public'}
2017-09-05 18:31:50,780 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog

2017-09-05 18:31:51,462 INFO sqlalchemy.engine.base.Engine COMMIT
2017-09-05 18:31:51,481 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind = 'r'
2017-09-05 18:31:51,482 INFO sqlalchemy.engine.base.Engine {'schema': u'public'}
2017-09-05 18:31:51,494 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2017-09-05 18:31:51,496 INFO sqlalchemy.engine.base.Engine {'name': u'resultadoAppend'}
2017-09-05 18:31:51,507 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2017-09-05 18:31:51,508 INFO sqlalchemy.engine.base.Engine {'name': u'resultadoAppend'}
2017-09-05 18:31:51,515 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c 

2017-09-05 18:31:51,693 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-09-05 18:31:51,729 INFO sqlalchemy.engine.base.Engine INSERT INTO "resultadoAppend" (id, nombre, precio, localidad, "tipoProducto") VALUES (%(id)s, %(nombre)s, %(precio)s, %(localidad)s, %(tipoProducto)s)
2017-09-05 18:31:51,731 INFO sqlalchemy.engine.base.Engine ({'localidad': 2, 'nombre': 'Nathan', 'precio': '$852.82', 'id': 1, 'tipoProducto': 8}, {'localidad': 2, 'nombre': 'Abramo', 'precio': '$249.03', 'id': 2, 'tipoProducto': 7}, {'localidad': 4, 'nombre': 'Ethelyn', 'precio': '$769.93', 'id': 3, 'tipoProducto': 2}, {'localidad': 7, 'nombre': 'Von', 'precio': '$32.94', 'id': 4, 'tipoProducto': 2}, {'localidad': 5, 'nombre': 'Latrena', 'precio': '$762.18', 'id': 5, 'tipoProducto': 10}, {'localidad': 3, 'nombre': 'Ancell', 'precio': '$511.36', 'id': 6, 'tipoProducto': 9}, {'localidad': 5, 'nombre': 'Boy', 'precio': '$416.98', 'id': 7, 'tipoProducto': 2}, {'localidad': 10, 'nombre': 'Holly-anne', 'precio

2017-09-05 18:31:52,434 INFO sqlalchemy.engine.base.Engine {'table_oid': 49439}
2017-09-05 18:31:52,439 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.consrc as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'c'
        
2017-09-05 18:31:52,440 INFO sqlalchemy.engine.base.Engine {'table_oid': 49439}
2017-09-05 18:31:52,443 INFO sqlalchemy.engine.base.Engine 
DROP TABLE "resultadoConcatenacion"
2017-09-05 18:31:52,444 INFO sqlalchemy.engine.base.Engine {}
2017-09-05 18:31:52,448 INFO sqlalchemy.engine.base.Engine COMMIT
2017-09-05 18:31:52,456 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "resultadoConcatenacion" (
	id BIGINT, 
	localidad FLOAT(53), 
	nombre TEXT, 
	precio TEXT, 
	producto FLOAT(53), 
	"tipoProducto" FLOAT(53)
)


2017-09-05 18:31:52,457 INFO sqlalchemy.engine.base.Engine {}
2017-09-0

2017-09-05 18:31:53,575 INFO sqlalchemy.engine.base.Engine {'table_oid': 49446}
2017-09-05 18:31:53,581 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        
2017-09-05 18:31:53,583 INFO sqlalchemy.engine.base.Engine {'table_oid': 49446}
2017-09-05 18:31:53,587 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.consrc as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                cons.conrelid = %(table_oid)s AND
                co